In [34]:
import os
import sys
import pandas as pd
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

<h1>Loading database</h1>

In [35]:
from pypots.benchpots.datasets import preprocess_physionet2012
physionet2012_dataset = preprocess_physionet2012(subset="all", rate=0.1)

2024-10-30 20:01:22 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2024-10-30 20:01:22 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2024-10-30 20:01:22 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2024-10-30 20:01:23 [INFO]: Loaded successfully!


<h2>Training data</h2>

<h3>Loading training dataset</h3>

In [36]:
train_X = physionet2012_dataset['train_X']

<h3>Divided into subgroups by gender and showing the percentage</h3>

In [37]:
distribution_gender_training = train_X['Gender'].value_counts(normalize=True) * 100
distribution_gender_training # 0: female, or 1: male

Gender
 1.0    56.589754
 0.0    43.358102
-1.0     0.052144
Name: proportion, dtype: float64

<h3>Divided into subgroups by ICUType and showing the percentage</h3>

In [38]:
distribution_ICUType_training = train_X['ICUType'].value_counts(normalize=True) * 100
distribution_ICUType_training

ICUType
3.0    35.562508
4.0    28.914092
2.0    20.975101
1.0    14.548299
Name: proportion, dtype: float64

<h3>Filtering only one age per patient</h3>

In [39]:
age_uniques_train = train_X.groupby('RecordID')['Age'].first().reset_index()
age_uniques_train.head()

,RecordID,Age
0,132540,76.0
1,132543,68.0
2,132547,64.0
3,132548,68.0
4,132555,74.0


<h3>Showing total occurrences of age</h3>

In [40]:
# 60% de 11988 (training set)
age_uniques_train.count()

RecordID    7671
Age         7671
dtype: int64

In [8]:
#first_values = train_X['Age'][::48] #first value of "Age" at every 48th index
#total_quantity_of_age_occurrences = first_values.count()
#total_quantity_of_age_occurrences

In [9]:
#first_values.head()

<h3>Dividing age into subgroup of people aged 65+</h3>

In [41]:
more_than_or_equal_to_65_training = age_uniques_train[age_uniques_train['Age'] >= 65]
# more_than_or_equal_to_65 = train_X[train_X['Age'] >= 65] 
#distribution_age_group_one = more_than_or_equal_to_65['Age'].count() 
#distribution_age_group_one
# age_uniques_more_than_or_equal_to_65 = more_than_or_equal_to_65.groupby('RecordID')['Age'].first().reset_index()
percentage_group_one_train = (more_than_or_equal_to_65_training.count() / age_uniques_train.count())*100
percentage_group_one_train

RecordID    54.373615
Age         54.373615
dtype: float64

<h3>Dividing age into subgroup of people under 65 years of age</h3>

In [42]:
less_than_65 = age_uniques_train[age_uniques_train['Age'] < 65]
# less_than_65 = train_X[train_X['Age'] < 65] 
#distribution_age_group_two = less_than_65['Age'].value_counts(normalize=True) * 100
#distribution_age_group_two

# age_uniques_less_than_65 = less_than_65.groupby('RecordID')['Age'].first().reset_index()
percentage_group_two_train = (less_than_65.count() / age_uniques_train.count()) * 100
percentage_group_two_train

RecordID    45.626385
Age         45.626385
dtype: float64

<h3>Filtering only one height and one weight per patient</h3>

In [43]:
filtered_train_X = train_X[(train_X['Height'] != -1) & (train_X['Weight'] != -1) & (train_X['Height'].notna()) & (train_X['Weight'].notna())] 

In [44]:
height_uniques_train = filtered_train_X.groupby('RecordID')['Height'].first().reset_index() 
height_uniques_train.head()

,RecordID,Height
0,132540,175.3
1,132543,180.3
2,132547,180.3
3,132548,162.6
4,132555,175.3


In [45]:
weight_uniques_train = filtered_train_X.groupby('RecordID')['Weight'].first().reset_index() 
weight_uniques_train.head()

,RecordID,Weight
0,132540,76.0
1,132543,84.6
2,132547,114.0
3,132548,87.0
4,132555,66.1


<h3>Calculate the BMI</h3>

In [46]:
def classify_BMI(BMI):
    if BMI < 18.5:
        return "Baixo peso"
    elif BMI >= 18.6 and BMI <= 24.9:
        return "Peso normal"
    elif BMI >= 25 and BMI <= 29.9:
        return "Sobrepeso"
    elif BMI >= 30 and BMI <= 34.9:
        return "Obesidade grau 1"
    elif BMI >= 35 and BMI <= 39.9:
        return "Obesidade grau 2"
    elif BMI >= 40:
        return "Obesidade grau 3"

In [47]:
height_uniques_train['Height'] = height_uniques_train['Height'] / 100 # Converting Height from cm to meters

In [48]:
bmi_data_train = pd.DataFrame(columns=['RecordID', 'Height', 'Weight', 'BMI', 'Classificacao'])
bmi_data_train["RecordID"] = height_uniques_train["RecordID"]
bmi_data_train["Height"] = height_uniques_train["Height"]
bmi_data_train["Weight"] = weight_uniques_train["Weight"]
bmi_data_train["BMI"] = weight_uniques_train["Weight"] / (height_uniques_train["Height"] ** 2)
bmi_data_train["Classificacao"] = bmi_data_train["BMI"].apply(classify_BMI)
bmi_data_train.head()
# Creating a new DataFrame to store relevant data
#bmi_data = filtered_train_X[['RecordID', 'Height', 'Weight', 'BMI']].reset_index(drop=True)

#bmi_data = pd.concat([height_uniques_train['RecordID'], height_uniques_train['Height'], weight_uniques_train['Weight']], sort=True)
#bmi_data = pd.merge(height_uniques_train[['RecordID', 'Height']], weight_uniques_train[['RecordID', 'Weight']], on='RecordID')
#bmi_data.head()
# bmi_data['BMI'] = weight_uniques_train['Weight'] / (height_uniques_train['Height'] ** 2)


,RecordID,Height,Weight,BMI,Classificacao
0,132540,1.753,76.0,24.731460,Peso normal
1,132543,1.803,84.6,26.024291,Sobrepeso
2,132547,1.803,114.0,35.068194,Obesidade grau 2
3,132548,1.626,87.0,32.906233,Obesidade grau 1
4,132555,1.753,66.1,21.509862,Peso normal


<h3>Percentage of BMI classification groups</h3>

In [49]:
bmi_data_train["Classificacao"].value_counts(normalize=True) * 100

Classificacao
Sobrepeso           35.019157
Peso normal         29.706258
Obesidade grau 1    18.288633
Obesidade grau 2     7.535121
Obesidade grau 3     6.666667
Baixo peso           2.784163
Name: proportion, dtype: float64

<h2>Validation data</h2>

<h3>Loading validation dataset </h3>

In [50]:
validation_X = physionet2012_dataset['val_X']

<h3>Divided into subgroups by gender and showing the percentage</h3>

In [51]:
distribution_gender_validation = validation_X['Gender'].value_counts(normalize=True) * 100
distribution_gender_validation # 0: female, or 1: male

Gender
 1.0    55.370177
 0.0    44.316997
-1.0     0.312826
Name: proportion, dtype: float64

<h3>Divided into subgroups by ICUType and showing the percentage</h3>

In [52]:
distribution_ICUType_validation = validation_X['ICUType'].value_counts(normalize=True) * 100
distribution_ICUType_validation

ICUType
3.0    37.539103
4.0    27.320125
2.0    21.584984
1.0    13.555787
Name: proportion, dtype: float64

<h3>Filtering only one age per patient</h3>

In [53]:
age_uniques_validation = validation_X.groupby('RecordID')['Age'].first().reset_index()
age_uniques_validation.head()

,RecordID,Age
0,132539,54.0
1,132541,44.0
2,132551,78.0
3,132597,66.0
4,132623,24.0


<h3>Showing total occurrences of age</h3>

In [54]:
age_uniques_validation.count()

RecordID    1918
Age         1918
dtype: int64

<h3>Dividing age into subgroup of people aged 65+</h3>

In [55]:
more_than_or_equal_to_65_validation = age_uniques_validation[age_uniques_validation['Age'] >= 65]
percentage_group_one_validation = (more_than_or_equal_to_65_training.count() / age_uniques_train.count())*100
percentage_group_one_validation

RecordID    54.373615
Age         54.373615
dtype: float64

<h3>Filtering only one height and one weight per patient</h3>

In [56]:
filtered_validation_X = validation_X[(validation_X['Height'] != -1) & (validation_X['Weight'] != -1) & (validation_X['Height'].notna()) & (validation_X['Weight'].notna())] 

In [57]:
height_uniques_validation = filtered_validation_X.groupby('RecordID')['Height'].first().reset_index() 
height_uniques_validation.head()

,RecordID,Height
0,132551,162.6
1,132597,137.2
2,132623,182.9
3,132637,170.2
4,132653,177.8


In [58]:
weight_uniques_validation = filtered_validation_X.groupby('RecordID')['Weight'].first().reset_index() 
weight_uniques_validation.head()

,RecordID,Weight
0,132551,48.4
1,132597,82.0
2,132623,78.0
3,132637,56.0
4,132653,91.3


In [59]:
height_uniques_validation['Height'] = height_uniques_validation['Height'] / 100 # Converting Height from cm to meters

In [60]:
bmi_data_validation = pd.DataFrame(columns=['RecordID', 'Height', 'Weight', 'BMI', 'Classificacao'])
bmi_data_validation["RecordID"] = height_uniques_train["RecordID"]
bmi_data_validation["Height"] = height_uniques_train["Height"]
bmi_data_validation["Weight"] = weight_uniques_train["Weight"]
bmi_data_validation["BMI"] = weight_uniques_train["Weight"] / (height_uniques_train["Height"] ** 2)
bmi_data_validation["Classificacao"] = bmi_data_validation["BMI"].apply(classify_BMI)
bmi_data_validation.head()

,RecordID,Height,Weight,BMI,Classificacao
0,132540,1.753,76.0,24.731460,Peso normal
1,132543,1.803,84.6,26.024291,Sobrepeso
2,132547,1.803,114.0,35.068194,Obesidade grau 2
3,132548,1.626,87.0,32.906233,Obesidade grau 1
4,132555,1.753,66.1,21.509862,Peso normal


<h3>Percentage of BMI classification groups</h3>

In [61]:
bmi_data_validation["Classificacao"].value_counts(normalize=True) * 100

Classificacao
Sobrepeso           35.019157
Peso normal         29.706258
Obesidade grau 1    18.288633
Obesidade grau 2     7.535121
Obesidade grau 3     6.666667
Baixo peso           2.784163
Name: proportion, dtype: float64

<h2>Test data</h2>

In [14]:
test_X = physionet2012_dataset['test_X']

In [15]:
age_uniques_test = test_X.groupby('RecordID')['Age'].first().reset_index()
age_uniques_test.count()

RecordID    2399
Age         2399
dtype: int64